## سدره ميرخان

Prepare libraries and data

In [1]:
!pip install Arabic-Stopwords

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 353 kB 34.1 MB/s 
     |████████████████████████████████| 126 kB 65.2 MB/s 


In [2]:
import pandas as pd
# here put every import you need e.g.
import nltk
import numpy as np
import matplotlib.pyplot as plt
import regex as re
from sklearn.feature_extraction.text import CountVectorizer
import arabicstopwords.arabicstopwords as stp
from nltk.stem.isri import ISRIStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras import layers,losses
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.svm import SVC  

Download data

In [3]:
# !wget 'https://drive.google.com/uc?export=download&id=1121dzWdV2ZIrQSSSLs0N4P5b3qRwg428' -O 'final_data.zip'

In [4]:
# !unzip final_data.zip

Prapere The Comparison Dictionary

In [31]:
model_comparison_table = {}

model_comparison_table['question_step_number'] = []
model_comparison_table['model_name'] = []
model_comparison_table['parameters'] = []
model_comparison_table['preprocessing_methods'] = []
model_comparison_table['accuracy'] = []

# Question [1]: Load the data

شرح ما يقوم به الكود

In [3]:
def drop_duplicate_empty(data):
    data.drop_duplicates(inplace = True,subset = 'text')
#     print("The total number of samples is after removing duplicates :",data.shape[0])
    data['label'].replace('  ', np.nan, inplace=True)
    data['text'].replace('  ', np.nan, inplace=True)
    data =data.dropna()
    return data

In [4]:
# your code here

def load_data(file):
    data = pd.read_csv(file)
    print("The total number of samples is :",data.shape[0])
#     print(data.head())
#     print()
    data = drop_duplicate_empty(data)
    print("The total number of samples is after removing empty and duplicates cells:",data.shape[0])
    return data


In [5]:
positivePattern = r"(p|P)(.)*"
irreleventPattern = r"(i|I)(.)*"
neutralPattern = r"(.)*(l|L)$"
labels = ['positive','negative','irrelevant','neutral']
def uniform_labels(x):
    x = str(x)
    if re.match(positivePattern,x):
        return re.sub(positivePattern,labels[0],x)
    elif re.match(irreleventPattern,x):
        return re.sub(irreleventPattern,labels[2],x)
    elif re.search(neutralPattern,x):
        return re.sub(neutralPattern,labels[3],x)
    return labels[1]
 
def uniform_dataframe_labels(data):
    data['label']= data['label'].apply(uniform_labels)


In [6]:
def get_tweets_targets_columns(data):
    x = data['text'] 
    y = data['label']
    return x,y

In [7]:

def get_postive_negative_data(data):
    df = data.copy()
    df =  df[df.label.isin([labels[0],labels[1]])]
    df['label_num'] = df.label.apply(lambda l:1 if l ==labels[0] else 0)
    # train_data_01 = train_data.query(f'label == {labels[0]} | label == {labels[1]}')
    print("the num of positive and negative in train is:",df.shape[0])
    return df

In [8]:
#     df["sentiments"].loc[df["sentiments"]=="positif"]=1.0
#     df["sentiments"].loc[df["sentiments"]=="negatif"]=0.0

In [9]:
def get_data(file):
    data =load_data(file)
    uniform_dataframe_labels(data)
    data_01 = get_postive_negative_data(data)
#     data_x,data_y = get_tweets_targets_columns(data_01)
    return data,data_01

## Train

In [20]:
train_data,train_data_01 =get_data('train.csv')

The total number of samples is : 33924
The total number of samples is after removing empty and duplicates cells: 24729
the num of positive and negative in train is: 10476


## Validation

In [10]:
valid_data,valid_data_01 =get_data('valid.csv')

The total number of samples is : 7269
The total number of samples is after removing empty and duplicates cells: 6723
the num of positive and negative in train is: 2873


## Test

In [11]:
test_data,test_data_01 =get_data('test.csv')

The total number of samples is : 7270
The total number of samples is after removing empty and duplicates cells: 6714
the num of positive and negative in train is: 2913


In [ ]:
# حال اختبار في حال وجودها

بعد كل خطوة ستكتب إجابة السؤال في حال وجوده أو ملاحظاتك أيضاً في حال وجودها.

# Question [2]: Baseline: Bag of Words with Logistic Regression

<font size="4">
 تمّ استخدام CountVectorizer لتحويل النص لـdata رقمية عن طريق تكرار الكلمة بالجملة، فنقوم بعمليتي fit و transform على داتا التدريب، , transform على داتا الاختبار، ومن ثم تم استخدام LogisticRegression لتنبؤ بالخرج ويلي هو 0 أو 1 ومن ثم اختبرنا الدقة 

In [ ]:
# do code
vectorizer = CountVectorizer()
vectorizer.fit(train_data_01.text)

X_train = vectorizer.transform(train_data_01.text)
X_valid = vectorizer.transform(valid_data_01.text)


In [ ]:
# test your model
classifier = LogisticRegression(max_iter =  200)
classifier.fit(X_train, train_data_01.label)
accuracy = classifier.score(X_valid, valid_data_01.label)
print("Accuracy:", accuracy)

Accuracy: 0.8064740689175078


In [32]:
model_comparison_table['question_step_number'].append("2")
model_comparison_table['model_name'].append("baseline logistic regression with bag of words")
model_comparison_table['parameters'].append("default")
model_comparison_table['preprocessing_methods'].append("none")
model_comparison_table['accuracy'].append(accuracy)

# Question [3]: Text Cleaning and Normalization on Bag of Words with Logistic Regression

Prapere The Comparison Dictionary for this step

In [ ]:
bag_logistic_comparison_table = {}

bag_logistic_comparison_table['question_step_number'] = []
bag_logistic_comparison_table['preprocessing_methods'] = []
bag_logistic_comparison_table['accuracy'] = []

In [13]:
def delete_mentions(tweet):
    tweet = re.sub(r"@\S+", '', tweet) #delete mentions
    return tweet

def delete_url(tweet):
    tweet = re.sub(r'https?\S+', '', tweet) #delete url
    return tweet

def delete_hashtag(tweet):
    tweet = re.sub(r'#\S+', '', tweet)
    return tweet

def delete_repeated_letter(tweet):
    pattern = r"([^\d٠-٩])\1{2,}"
    tweet = re.sub(pattern,r"\1",tweet)
    return tweet

diff= ord('٠')-ord('0')
def numbers_uniform(tweet):
    tweet = re.sub(r"([٠-٩])", lambda match: chr(ord(match.group(0))-diff),tweet)
    return tweet

num_pattern = r"[\d٠-٩]+"
def replace_numbers(tweet):
    output =  re.sub(num_pattern,"*",tweet)
    return output

def del_numbers(tweet):
    output =  re.sub(num_pattern,"",tweet)
    return output

pnc_pattern = r"[\.!$%&()+?#_,<>/-:;=\\،^`{|}~\*؟]+"
def replace_symbols(tweet):
    output =  re.sub(pnc_pattern,"",tweet)
    return output

emj_pattern =r"[^\p{L}\p{M}\p{N}\p{P}\p{Z}\p{Cf}\p{Cs}\s]"
def delete_emojies(tweet):
    output =  re.sub(emj_pattern,"",tweet)
    return output

def delete_all_not_arabic(tweet):
    pattern = r"([\u0600-\u06FF]+|[^\p{L}\p{M}\p{N}\p{P}\p{Z}\p{Cf}\p{Cs}\s]+|[@!$%&()+?,./:;=\\،^`{|}~\*]+|[\d\.٠-٩]+|#\S+)|(\S*)"
    tweet = re.sub(pattern, lambda match: match.groups()[0],tweet) 
    return tweet

def del_stop_words(text):
    return " ".join([w for w in text.split() if not stp.is_stop(w)])

st = ISRIStemmer()
def lemmatization(text):
    return " ".join([st.stem(w) for w in text.split()])

def uniform_hamza(text):
    output = re.sub(r"ئ|ؤ","ء",text)
    return output

def uniform_alef(text):
    output = re.sub(r"إ|أ|آ","ا",text)
    return output

def del_tatweel(text):
    output = re.sub( r"ـ","",text)
    return output

def del_tashkil(text):
    tashkil_pattern = r"[\u064B-\u0652]"
    output = re.sub(tashkil_pattern ,"",text)
    return output

def del_space(text):
    output = re.sub(r"(\s)\1+",r"\1",text)
    return output

In [14]:
clean= {  "remove_url" : "3.1",
                "remove_mentions" : "3.1",
                "remove_hashtags" : "3.2",
                "remove_repeated":"3.3",
                "uniform_nums" : "3.4.1",
                "replace_nums" :"3.4.2",
                "remove_nums":"3.4.3",
                "remove_all_non_arabic":"3.5.1",
                "remove_punctuation": "3.5.2",
                "remove_emojies":"3.5.3",
                "remove_stop_words":"3.6", 
                "lemmatization":"3.7",
                "uniform_hamza" : "3.8.1",
                "uniform_alef" : "3.8.2",
                "tatweel" : "3.8.3",
                "tashkil" : "3.8.4",
                "remove_space" :"3.9"
             }

clean_list = [ "remove_url" ,"remove_mentions" ,"remove_hashtags","remove_repeated","uniform_nums","replace_nums",
                "remove_nums","remove_all_non_arabic","remove_punctuation","remove_emojies",
                "remove_stop_words", "lemmatization","uniform_hamza","uniform_alef" ,"tatweel","tashkil" ,
              "remove_space" ]

In [15]:
def preprocess(tweet,clean_list= clean_list):
  
    if "remove_url" in clean_list:
        tweet = delete_url(tweet)
#         print(clean["remove_url"])
    if "remove_mentions" in clean_list:
        tweet =delete_mentions(tweet)
#         print(clean["remove_mentions"])
    if "remove_hashtags" in clean_list:
        tweet =delete_hashtag(tweet)
#         print(clean["remove_hashtags"])
    if "remove_repeated" in clean_list:
        tweet =delete_repeated_letter(tweet)
#         print(clean["remove_repeated"])
        
    if "uniform_nums" in clean_list:
        tweet = numbers_uniform(tweet)
#         print(clean["uniform_nums"])
    if "replace_nums" in clean_list:
        tweet =  replace_numbers(tweet)
#         print(clean["replace_nums"])
    if "remove_nums" in clean_list:
        tweet =del_numbers(tweet)
#         print(clean["remove_nums"])
    if "remove_punctuation" in clean_list:
        tweet = replace_symbols(tweet)
#         print(clean["remove_punctuation"])
    if "remove_emojies" in clean_list:
        tweet = delete_emojies(tweet)
#         print(clean["remove_emojies"])
    if "remove_all_non_arabic" in clean_list:
        tweet = delete_all_not_arabic(tweet)
#         print(clean["remove_all_non_arabic"])
        
    if  "uniform_hamza" in clean_list:
        tweet = uniform_hamza(tweet)
#         print(clean["uniform_hamza"])
    if "uniform_alef" in clean_list:
        tweet =  uniform_alef(tweet)
#         print(clean["uniform_alef"])
    if "tatweel" in clean_list:
        tweet = del_tatweel(tweet)
#         print(clean["tatweel"])
    if "tashkil" in clean_list:
        tweet = del_tashkil(tweet)
#         print(clean["tashkil"])
    if "remove_space" in clean_list:
        tweet = del_space(tweet)
#         print(clean["remove_space"])
        
    if "remove_stop_words" in clean_list:
        tweet = del_stop_words(tweet)
#         print(clean["remove_stop_words"])
    if "lemmatization" in clean_list:
        tweet = lemmatization(tweet)
#         print(clean["lemmatization"])
    
    return tweet

In [16]:
def store_results(clean_list,accuracy):
    preprocessing_methods = ""
    question_step_number= ""
    preprocessing_methods = " + ".join(clean_list)
    question_step_number= clean[clean_list[-1]]
    
    bag_logistic_comparison_table['question_step_number'].append(question_step_number)
    bag_logistic_comparison_table['preprocessing_methods'].append(preprocessing_methods)
    bag_logistic_comparison_table['accuracy'].append(accuracy)

In [17]:
def drop_duplicate_empty_for_clean_data(data,c="cleaned"):
    data.drop_duplicates(inplace = True,subset =c )
    data[c].replace('  ', np.nan, inplace=True)
    data =data[data[c].notna()]
    return data

<font size="4">
يأخذ التابع عمليات التنظيف التي سيقوم بها وينفذها على داتا التدريب وداتا الاختبار ونقوم بعمليتي fit و transform على داتا التدريب، , transform على داتا الاختبار ومن ثم نختبر دقة الـlogistic regression ونخزنها بdictionary المقارنة، وسنستدعي هذا التابع من أجل كل عملية تنظيف بهذا القسم

In [ ]:

def classify_on_cleaned_data(clean):
    # do code
#     train_data_01_x_cleaned =list(map(lambda tweet: preprocess(tweet, clean),train_data_01_x))
#     valid_data_01_x_cleaned =list(map(lambda tweet: preprocess(tweet, clean),valid_data_01_x))
    train = train_data_01.copy()
    valid = valid_data_01.copy()
    train["cleaned"] =train.text.apply(lambda tweet: preprocess(tweet, clean))
    valid["cleaned"] =valid.text.apply(lambda tweet: preprocess(tweet, clean))

    train = drop_duplicate_empty_for_clean_data(train)
    valid = drop_duplicate_empty_for_clean_data(valid)
    
    vectorizer = CountVectorizer()
    classifier = LogisticRegression(max_iter =  200)
    
    X_train = vectorizer.fit_transform(train.cleaned)
    X_valid = vectorizer.transform(valid.cleaned)

    # test your model
    classifier.fit(X_train, train.label_num)
    accuracy = classifier.score(X_valid, valid.label_num)
    
    store_results(clean,accuracy)
#     print(train.cleaned.values[:5])
    print("Accuracy:", accuracy)

## [3.1] 

In [ ]:
classify_on_cleaned_data( ["remove_url" ,"remove_mentions"])

Accuracy: 0.8044614848379226


## [3.2] 

<font size="4">
إنّ حذف الهاشتاغات أنقص الدقة قليلا ولذلك لأنها قد تحوي معلومات مهمة 
أتوقع أن إبقاء هذه الـهاشتاغات سيتعطي دقة جيدة في تصنيف بيانات جديدة
أخذت في فترة زمنية مختلفة عن الفترة التي تم جمع البيانات فيها

In [ ]:
classify_on_cleaned_data(["remove_hashtags"])

Accuracy: 0.801811215604319


## [3.3] 

In [ ]:
classify_on_cleaned_data(["remove_repeated"])

Accuracy: 0.8054298642533937


## [3.4] 

### [3.4.1] 

In [ ]:
classify_on_cleaned_data(["uniform_nums"])

Accuracy: 0.8064740689175078


### [3.4.2] 

In [ ]:
classify_on_cleaned_data(["replace_nums"]) 

Accuracy: 0.8085624782457361


### [3.4.3] 

In [ ]:
classify_on_cleaned_data(["remove_nums"])

Accuracy: 0.8050817960320222


## [3.5] 

### [3.5.1] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_all_non_arabic"])

Accuracy: 0.8030672708260718


### [3.5.2] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_punctuation"])

Accuracy: 0.8057103064066853


### [3.5.3] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies"])

Accuracy: 0.8082144100243648


## [3.6] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","remove_stop_words"])

Accuracy: 0.8006273963053329


## [3.7] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","lemmatization"])

Accuracy: 0.7936563262460787


## [3.8] 

### [3.8.1] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","uniform_hamza"])

0.8078663418029934

### [3.8.2] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","uniform_alef"])


0.8033414549251653

### [3.8.3] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","tatweel"])


0.8091922005571031

### [3.8.4] 

In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","tatweel","tashkil"])

0.8102367688022284

## [3.9] 

<font size="4">
نلاحظ أن حذف الفراغات أنقص الدقة والأسباب مجهولة



In [ ]:
classify_on_cleaned_data(["replace_nums","remove_emojies","tatweel","tashkil","remove_space"])

Accuracy: 0.8084958217270195


## [3.10]

## Print Table

In [ ]:
df = pd.DataFrame(bag_logistic_comparison_table)
df

,question_step_number,preprocessing_methods,accuracy
0,3.1,remove_url + remove_mentions,0.804461
1,3.2,remove_hashtags,0.801811
2,3.3,remove_repeated,0.805430
3,3.4.1,uniform_nums,0.806474
4,3.4.2,replace_nums,0.808562
5,3.4.3,remove_nums,0.805082
6,3.5.1,replace_nums + remove_all_non_arabic,0.803067
7,3.5.2,replace_nums + remove_punctuation,0.805710
8,3.5.3,replace_nums + remove_emojies,0.808214
9,3.6,replace_nums + remove_emojies + remove_stop_words,0.800627


## Your best model
إن أفضل العمليات هي حذف استبدال الأرقام بنجمة وحذف الايموجيز وحذف التشكيل والتطويل 

In [22]:
clean = ["replace_nums","remove_emojies","tatweel","tashkil"]
train = train_data_01.copy()
test = test_data_01.copy()
train["cleaned"] =train.text.apply(lambda tweet: preprocess(tweet, clean))
test["cleaned"] =test.text.apply(lambda tweet: preprocess(tweet, clean))

train = drop_duplicate_empty_for_clean_data(train)
test = drop_duplicate_empty_for_clean_data(test)

In [ ]:
vectorizer = CountVectorizer()
classifier = LogisticRegression(max_iter =  200)

X_train = vectorizer.fit_transform(train.cleaned)
X_test = vectorizer.transform(test.cleaned)

# test your model
classifier.fit(X_train, train.label_num)
accuracy = classifier.score(X_test, test.label_num)

print("Accuracy:", accuracy)

Accuracy: 0.8084449021627188


In [33]:
model_comparison_table['question_step_number'].append("3")
model_comparison_table['model_name'].append("logistic regression with bag of words")
model_comparison_table['parameters'].append("defualt")
model_comparison_table['preprocessing_methods'].append("replace_nums + remove_emojies + tatweel + tashkil")
model_comparison_table['accuracy'].append(accuracy)

# Question [4]: TF-IDF with Logistic Regression

Prapere The Comparison Dictionary for this step

In [ ]:
tf_idf_comparison_table = {}

tf_idf_comparison_table['question_step_number'] = []
tf_idf_comparison_table['model_name'] = []
tf_idf_comparison_table['parameters'] = []
tf_idf_comparison_table['accuracy'] = []

In [ ]:
def store_result_tf_idf(question_step_number,accuracy,model_name = "logistic regression with TF_IDF",
                        parameters="default parameters"):
    tf_idf_comparison_table['question_step_number'].append(question_step_number)
    tf_idf_comparison_table['model_name'].append(model_name)
    tf_idf_comparison_table['parameters'].append(parameters)
    tf_idf_comparison_table['accuracy'].append(accuracy)

In [ ]:
valid = valid_data_01.copy()
valid["cleaned"] =valid.text.apply(lambda tweet: preprocess(tweet, clean))
valid = drop_duplicate_empty_for_clean_data(valid)

<font size="4">
يأخذ التابع data التدريب والاختبار(valid /test) حسب الطلب، وباراميترات الـTfidf ومتحول لمعرفة الdictionary الذي سيخزن به، التابع شبيه بما قام فيه التابع بالقسم الثاني ولكن هنا Tfidf بدلا من Countvectorizer

In [ ]:
def tfidf(test,train=train, min_df=1 ,max_df=1,ngram_range=(1, 2),store="valid"):
 
    tfidf_vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    classifier = LogisticRegression(max_iter =  200)

    X_train = tfidf_vectorizer.fit_transform(train.cleaned)
    X_test = tfidf_vectorizer.transform(test.cleaned)

    # test your model
    classifier.fit(X_train, train.label_num)
    accuracy = classifier.score(X_test, test.label_num)
    parameters = f"min_df={min_df} max_df={max_df} ngram={ngram_range}"
    if store == "valid":
        store_result_tf_idf(question_step_number ="4.1",accuracy= accuracy,parameters = parameters)

    print("Accuracy:", accuracy)
    return accuracy

## [4.1] 

In [ ]:
tfidf(valid)

Accuracy: 0.7653203342618384


In [ ]:
# tfidf(clean = clean,test=test_data_01)

## [4.2] 

In [ ]:
tfidf(test=valid,min_df=2, max_df=0.5, ngram_range=(1, 2))

Accuracy: 0.7802924791086351


In [ ]:
tfidf(test=valid,min_df=0.2, max_df=0.5, ngram_range=(1, 2))

Accuracy: 0.5825208913649025


In [ ]:
tfidf(test=valid,min_df=3, max_df=0.5, ngram_range=(1, 2))

Accuracy: 0.7733286908077994


In [ ]:
tfidf(test=valid,min_df=0.4, max_df=0.8, ngram_range=(1, 2))

Accuracy: 0.5362116991643454


In [ ]:
tfidf(test=valid,min_df=0.3, max_df=0.8, ngram_range=(1, 3))

Accuracy: 0.5898328690807799


In [ ]:
tfidf(test=valid,min_df=2, max_df=0.8, ngram_range=(1, 3))

Accuracy: 0.7827298050139275


In [ ]:
tfidf(test=valid,min_df=0.02, max_df=0.5, ngram_range=(1, 3))

Accuracy: 0.6368384401114207


In [ ]:
tfidf(test=valid,min_df=0.05, max_df=0.5, ngram_range=(1, 3))

Accuracy: 0.5988857938718662


In [ ]:
tfidf(test=valid,min_df=0.08, max_df=0.8, ngram_range=(1, 3))

Accuracy: 0.6002785515320335


In [ ]:
tfidf(test=valid,min_df=0.015, max_df=0.5, ngram_range=(1, 3))

Accuracy: 0.6511142061281338


In [ ]:
tfidf(test=valid,min_df=0.0005, max_df=0.5, ngram_range=(1, 3))

Accuracy: 0.7614902506963789


In [ ]:
tfidf(test=valid,min_df=0, max_df=0.5, ngram_range=(1, 3))

Accuracy: 0.7914345403899722


In [ ]:
tfidf(test=valid,min_df=0, max_df=0.4, ngram_range=(1, 3))

Accuracy: 0.7907381615598886


In [ ]:
tfidf(test=valid,min_df=0, max_df=0.7, ngram_range=(1, 3))

Accuracy: 0.7903899721448467


In [ ]:
tfidf(test=valid,min_df=0, max_df=0.55, ngram_range=(1, 3))

Accuracy: 0.7914345403899722


In [ ]:
tfidf(test=valid,min_df=0, max_df=0.5, ngram_range=(1, 2))

Accuracy: 0.7910863509749304


In [ ]:
tfidf(test=valid,min_df=0, max_df=0.5, ngram_range=(1, 4))

Accuracy: 0.7896935933147632


0.7896935933147632

In [ ]:
tfidf(test=valid,min_df=0, max_df=0.5, ngram_range=(1, 3))

## [4.3] 

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train.cleaned)
X_valid = vectorizer.transform(valid.cleaned)

In [ ]:
def classify(classifier):
  classifier.fit(X_train, train.label_num)
  accuracy = classifier.score(X_valid, valid.label_num)
  print("Accuracy:", accuracy)
  return accuracy

In [ ]:
accuracy = classify( SVC())

Accuracy: 0.5698697641675466


In [ ]:
accuracy = classify(SVC(kernel ="sigmoid"))

Accuracy: 0.5181274199225625


In [ ]:
accuracy = classify( SVC(degree = 5))

Accuracy: 0.5698697641675466


In [ ]:
accuracy = classify(SVC(kernel ="poly"))

Accuracy: 0.518479408658923


<font size="4">
نلاحظ أنها لم تعطي نتائج جيدة




## Print Table

In [ ]:
df = pd.DataFrame(tf_idf_comparison_table)
df

,question_step_number,model_name,parameters,accuracy
0,4.1,logistic regression with TF_IDF,"min_df=1 max_df=1 ngram=(1, 2)",0.765320
1,4.1,logistic regression with TF_IDF,"min_df=2 max_df=0.5 ngram=(1, 2)",0.780292
2,4.1,logistic regression with TF_IDF,"min_df=0.2 max_df=0.5 ngram=(1, 2)",0.582521
3,4.1,logistic regression with TF_IDF,"min_df=3 max_df=0.5 ngram=(1, 2)",0.773329
4,4.1,logistic regression with TF_IDF,"min_df=0.4 max_df=0.8 ngram=(1, 2)",0.536212
5,4.1,logistic regression with TF_IDF,"min_df=0.3 max_df=0.8 ngram=(1, 3)",0.589833
6,4.1,logistic regression with TF_IDF,"min_df=2 max_df=0.8 ngram=(1, 3)",0.782730
7,4.1,logistic regression with TF_IDF,"min_df=0.02 max_df=0.5 ngram=(1, 3)",0.636838
8,4.1,logistic regression with TF_IDF,"min_df=0.05 max_df=0.5 ngram=(1, 3)",0.598886
9,4.1,logistic regression with TF_IDF,"min_df=0.08 max_df=0.8 ngram=(1, 3)",0.600279


## Your best model
ما هو النموذج الأفضل؟ اكتبه هنا

In [ ]:
# test the model
min_df=0
max_df=0.5
ngram_range=(1, 3)
accuracy=tfidf(test=test,min_df=min_df, max_df=max_df, ngram_range=ngram_range,store="")


Accuracy: 0.8039821489872984


In [34]:

parameters = f"min_df={min_df} max_df={max_df} ngram={ngram_range}"

model_comparison_table['question_step_number'].append("4")
model_comparison_table['model_name'].append("logistic regression with tf-idf")
model_comparison_table['parameters'].append(parameters)
model_comparison_table['preprocessing_methods'].append(" + ".join(clean))
model_comparison_table['accuracy'].append(accuracy)

# Question [5]: Deep NN

Prapere The Comparison Dictionary for this step

In [ ]:
dnn_comparison_table = {}

dnn_comparison_table['question_step_number'] = []
dnn_comparison_table['model_name'] = []
dnn_comparison_table['parameters'] = []
dnn_comparison_table['accuracy'] = []

## [5.1] 
شرح ما يقوم به الكود


In [23]:
from keras.backend import clear_session
# clear_session()

In [35]:
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(train.cleaned).toarray()
X_valid = vectorizer.transform(valid.cleaned).toarray()
X_test = vectorizer.transform(test.cleaned).toarray()

y_train = train.label_num.values
y_valid = valid.label_num.values
y_test = test.label_num.values

In [ ]:
# do code
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

history = model.fit(X_train,y_train,
                     epochs=100,
                     verbose=False,
#                      validation_split=0.2,
#                     validation_data = (X_valid,y_valid),
                     batch_size=10)

loss, accuracy = model.evaluate(X_train,y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test,y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

<class 'numpy.ndarray'>
Training Accuracy: 0.9994
Testing Accuracy:  0.7872


In [ ]:
dnn_comparison_table['question_step_number'].append("5.1")
dnn_comparison_table['model_name'].append("Fully Connected DNN")
dnn_comparison_table['parameters'].append("loss='binary_crossentropy', optimizer='adam', Dense neurons = 10")
dnn_comparison_table['accuracy'].append(accuracy)

## [5.2] 

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train.cleaned)

X_train = tokenizer.texts_to_sequences(train.cleaned)
X_test = tokenizer.texts_to_sequences(test.cleaned)
X_valid = tokenizer.texts_to_sequences(valid.cleaned)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


maxlen = 150

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
X_valid = pad_sequences(X_valid, padding='post', maxlen=maxlen)

In [ ]:
# do code
embedding_dim = 50
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

history = model.fit(X_train,y_train,
                     epochs=100,
                     verbose=False,
#                      validation_split=0.2,
                    validation_data = (X_valid,y_valid),
                     batch_size=10)


In [ ]:
loss, accuracy = model.evaluate(X_train,y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test,y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9985
Testing Accuracy:  0.7652


In [ ]:
# do code
embedding_dim = 50
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

history = model.fit(X_train,y_train,
                     epochs=100,
                     verbose=False,
#                      validation_split=0.2,
                    validation_data = (X_valid,y_valid),
                     batch_size=10)


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x7f81fb6083a0>
Traceback (most recent call last):
  File "/usr/lib/python3.8/weakref.py", line 345, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


In [ ]:
loss, accuracy = model.evaluate(X_train,y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test,y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.5135
Testing Accuracy:  0.5125


In [ ]:
dnn_comparison_table['question_step_number'].append("5.2")
dnn_comparison_table['model_name'].append("Fully Connected DNN with Dropout")
dnn_comparison_table['parameters'].append("loss='binary_crossentropy', optimizer='adam', Dense neurons = 10")
dnn_comparison_table['accuracy'].append(accuracy)

### Your best model

In [36]:

model_comparison_table['question_step_number'].append("5.2")
model_comparison_table['model_name'].append("Fully Connected DNN")
model_comparison_table['parameters'].append("loss='binary_crossentropy', optimizer='adam', Dense neurons = 10, embedding layer")
model_comparison_table['preprocessing_methods'].append(" + ".join(clean))
model_comparison_table['accuracy'].append(accuracy)

## [5.3] 

In [ ]:
embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                    validation_data = (X_valid,y_valid),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.9294
Testing Accuracy:  0.7583


In [ ]:
dnn_comparison_table['question_step_number'].append("5.3")
dnn_comparison_table['model_name'].append("CNN")
dnn_comparison_table['parameters'].append("loss='binary_crossentropy', optimizer='adam', 1 Conv with 128 filters with size 5*5")
dnn_comparison_table['accuracy'].append(accuracy)

## [5.4] 

In [ ]:
model = Sequential()

# Embedding layer
model.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))

model.add(layers.LSTM(64, return_sequences=False,  dropout=0.1, recurrent_dropout=0.1))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
                    epochs=10,
                    verbose=False,
                     validation_data = (X_valid,y_valid),
                    batch_size=10)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.5135
Testing Accuracy:  0.5125


In [ ]:
dnn_comparison_table['question_step_number'].append("5.4")
dnn_comparison_table['model_name'].append("LSTM")
dnn_comparison_table['parameters'].append("loss='binary_crossentropy', optimizer='adam', Lstm neurons = 64")
dnn_comparison_table['accuracy'].append(accuracy)

### Your best model

In [ ]:
# test the model


In [37]:
model_comparison_table['question_step_number'].append("5.4")
model_comparison_table['model_name'].append("LSTM")
model_comparison_table['parameters'].append("loss='binary_crossentropy', optimizer='adam', Lstm neurons = 64")
model_comparison_table['preprocessing_methods'].append(" + ".join(clean))
model_comparison_table['accuracy'].append(accuracy)

## [5.5] 

## Print Table

In [ ]:
import pandas as pd

df = pd.DataFrame(dnn_comparison_table)
df

,question_step_number,model_name,parameters,accuracy
0,5.1,Fully Connected DNN,"loss='binary_crossentropy', optimizer='adam', ...",0.7872
1,5.2,Fully Connected DNN with Dropout,"loss='binary_crossentropy', optimizer='adam', ...",0.5125
2,5.4,LSTM,"loss='binary_crossentropy', optimizer='adam', ...",0.5125
3,5.3,CNN,"loss='binary_crossentropy', optimizer='adam', ...",0.7583


# Question [6]: Train on all Classes

## [6.1] 
شرح ما يقوم به الكود


In [ ]:
from sklearn import preprocessing
# label_encoder object knows how to understand word labels.
def encoding(data): 
    label_encoder = preprocessing.LabelEncoder()
    # Encode labels in column 'Country'. 
    data["label"]= label_encoder.fit_transform(data["label"]) 
    print(data.head())
    return data

In [24]:
def convert_to_num(l):
  if l== labels[0]:
    return 0
  elif l== labels[1]:
    return 1
  elif l== labels[2]:
    return 2
  else:
    return 3

In [25]:
# do code
clean = ["replace_nums","remove_emojies","tatweel","tashkil"]

train_data["label"] = train_data.label.apply(convert_to_num)
# valid_data["label_num"] = valid_data.label.apply(convert_to_num)
test_data["label"] = test_data.label.apply(convert_to_num)
# train_data = encoding(train_data)
# valid_data = encoding(valid_data)
# test_data = encoding(test_data)

train_data["text"] =train_data.text.apply(lambda tweet: preprocess(tweet, clean))
test_data["text"] =test_data.text.apply(lambda tweet: preprocess(tweet, clean))
# valid_data["cleaned"] =valid_data.text.apply(lambda tweet: preprocess(tweet, clean))

train_data = drop_duplicate_empty_for_clean_data(train_data,c="text")
# valid_data = drop_duplicate_empty_for_clean_data(valid_data)
test_data = drop_duplicate_empty_for_clean_data(test_data,c="text")
    
vectorizer = CountVectorizer()

X_train = vectorizer.fit_transform(train_data.text)
# X_valid = vectorizer.transform(valid_data.cleaned).toarray()
X_test = vectorizer.transform(test_data.text)

y_train = train_data.label.values
# y_valid = valid.label_num.values
y_test = test_data.label.values

In [26]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data.text)

X_train = tokenizer.texts_to_sequences(train_data.text)
X_test = tokenizer.texts_to_sequences(test_data.text)
# X_valid = tokenizer.texts_to_sequences(valid.cleaned)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index


maxlen = 150

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)
# X_valid = pad_sequences(X_valid, padding='post', maxlen=maxlen)

In [27]:
# do code
embedding_dim = 50
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.compile(loss=losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
# model.summary()

history = model.fit(X_train,y_train,
                     epochs=100,
                     verbose=False,
                     validation_split=0.2,
                    # validation_data = (X_valid,y_valid),
                     batch_size=10)


In [30]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.8796
Testing Accuracy:  0.5920


In [38]:
model_comparison_table['question_step_number'].append("6.1")
model_comparison_table['model_name'].append("DNNwith all classes")
model_comparison_table['parameters'].append("loss='spase_categorical_crossentropy', optimizer='adam' ,Dense with 10 neorons ,embedding layer")
model_comparison_table['preprocessing_methods'].append(" + ".join(clean))
model_comparison_table['accuracy'].append(accuracy)

## [6.2] 


# Final Results

In [39]:
df = pd.DataFrame(model_comparison_table)
df

,question_step_number,model_name,parameters,preprocessing_methods,accuracy
0,2,baseline logistic regression with bag of words,default,none,0.806474
1,3,logistic regression with bag of words,defualt,replace_nums + remove_emojies + tatweel + tashkil,0.808445
2,4,logistic regression with tf-idf,"min_df=0 max_df=0.5 ngram=(1, 3)",replace_nums + remove_emojies + tatweel + tashkil,0.803982
3,5.2,Fully Connected DNN,"loss='binary_crossentropy', optimizer='adam', ...",replace_nums + remove_emojies + tatweel + tashkil,0.512500
4,5.4,LSTM,"loss='binary_crossentropy', optimizer='adam', ...",replace_nums + remove_emojies + tatweel + tashkil,0.512500
5,6.1,DNNwith all classes,"loss='spase_categorical_crossentropy', optimiz...",replace_nums + remove_emojies + tatweel + tashkil,0.592046


In [40]:
df.to_csv("سدره ميرخان.csv", index=False)